# Introduction to *pyDeltaRCM*

*pyDeltaRCM* is a computationally efficient, free and open source, and easy-to-customize numerical delta model based on the original DeltaRCM model design ([MATLAB deltaRCM model](https://csdms.colorado.edu/wiki/Model:DeltaRCM) by Man Liang; [Liang et al., 2015](https://doi.org/10.5194/esurf-3-67-2015)). *pyDeltaRCM* delivers improved model stability and capabilities, infrastructure to support exploration with minimal boilerplate code, and establishes an approach to extending model capabilities that ensures reproducible and comparable studies. 

<figure>
<img src="https://camo.githubusercontent.com/f11f16e33a9d33272c5d9aa549083285a2063ed6d3dc4307bc16a1e061b50f19/68747470733a2f2f64656c746172636d2e6f72672f707944656c746152434d2f5f696d616765732f636f7665722e706e67" width="500"/>
<figcaption> Figure:  Weighted random walks for 20 water parcels </figcaption>
</figure>

<br>

## Aims of this notebook

In this notebook, we will attempt to:

- Showcase some examples of what can be done with the model
- Provide an overview of how the *pyDeltaRCM* model is structured
- Walk through the use and configuration of standard model parameters
- Run a "mini" version of the model
- Load some "full-size" model results and examine them

<br>

### Useful Links and Resources

[Documentation](https://deltarcm.org/pyDeltaRCM/index.html) for the *pyDeltaRCM* model is available online and includes instructions on how to [install the model locally](https://deltarcm.org/pyDeltaRCM/meta/installing.html), [a user guide](https://deltarcm.org/pyDeltaRCM/guides/user_guide.html), [example scripts](https://deltarcm.org/pyDeltaRCM/examples/index.html), and much more.

<br>

More generally, for information on the Python programming language we suggest you explore their [documentation](https://www.python.org/doc/).
To manage local programming environments and package installations we recommend checking out [Anaconda](https://www.anaconda.com/).
For good scientific programming practicies, we encourage you to check out some of the material CSDMS has put out, including the "Level Up!" [webinar](https://csdms.colorado.edu/wiki/Webinars) series.



# The DeltaRCM Modeling Framework



The original DeltaRCM model was conceived as a "reduced-complexity" model capable of simulating the growth and evolution of a river delta landscape.
Simulation of the movement of water and sediment is discretized into "parcels" of fluid which move through the domain via a weighted random walk.
As these parcels walk about, they modify the flow fields or topography according to empirical rules, derived from physical relationships known to govern hydrodynamics and sediment transport.
For more details on the random walks, we refer you to the [original DeltaRCM paper](https://doi.org/10.5194/esurf-3-67-2015).

<br>

<figure>
<img src="https://raw.githubusercontent.com/DeltaRCM/pyDeltaRCM/develop/assets/private/liang_2015_fig1fig2.png" width="900"/>
<figcaption> NOTE: Figures are from Liang et al., 2015a, ESurf </figcaption>
</figure>

<br>

<figure>
<img src="https://github.com/DeltaRCM/pyDeltaRCM/blob/gh-pages/pyplots/water_tools/run_water_iteration.hires.png?raw=true" width="900"/>
<figcaption> Figure:  example of random walk through initial domain (top) and after delta development (bottom) </figcaption>
</figure>


# What can the model do?

To help motivate our use of this model. Let's have a look at what it actually can do.

<br>

Simulate the effect different sediment compositions (here as fraction of sand $f_{sand}$) have on deltaic evolution:
<figure>
<img src="https://utexas.box.com/shared/static/fmwtqeg1kiorr0m6vhyw6yemc6jugrsv.gif" width="900"/>
</figure>

<br>

Simulate delta evolution as sea levels rise:
<figure>
<img src="https://utexas.box.com/shared/static/wfpwxg73atkb8j2lfq23rgbzqnn1tplr.gif" width="600"/>
</figure>

<figure>
<img src="https://utexas.box.com/shared/static/txzytp8h07mqtq2nv0iy5neiuvsaevei.gif" width="600"/>
</figure>

<br>

Simulate the effects of vegetation ([Lauzon et al 2018](https://doi.org/10.1029/2018GL079405)):
<figure>
<img src="https://utexas.box.com/shared/static/qcp9y2dhujtmozt9467kigezm4e5hkvh.gif" width="900"/>
</figure>

<br>

Simulate the effects of ice ([Lauzon et al 2019](https://doi.org/10.1029/2019GL082792)):
<figure>
<img src="https://utexas.box.com/shared/static/k9v2h5j9jsgw4lao8vtuy0q2q9e0dg5m.gif" width="900"/>
</figure>

<br>

Simulate the effects of permafrost ([Lauzon et al 2019](https://doi.org/10.1029/2019GL082792)):
<figure>
<img src="https://utexas.box.com/shared/static/era1petej6hcwzi48zhjcju62rjljiui.gif" width="900"/>
</figure>

<br>

# Install *pyDeltaRCM*

We will use `pip` to install the package into the Colab environment.

To use *pyDeltaRCM* on your local machine you can do the same `pip` installation. If you wish to join our developer community, we encourage you to download the full repository and install the package from source, developer installation instructions are available [here](https://deltarcm.org/pyDeltaRCM/meta/installing.html#developer-installation).

In [ ]:
%%capture

!pip install pydeltarcm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Running the model in 5 lines of code

The *pyDeltaRCM* model operations are split into many different "methods" (i.e., functions) that represent a logical step in the model routine. 
We've made an effort to name these methods descriptively, so that the high level functionality of each method is immediately clear, but all of the methods are also individually documented (e.g., [water methods are documented here](https://deltarcm.org/pyDeltaRCM/reference/water_tools/index.html#public-api-methods-attached-to-model)).

---
* `init_domain`
* `init_output_infrastructure`
* `update`
  * `solve_water_and_sediment_timestep`
    * `route_water`
      * `init_water_iteration`
      * `run_water_iteration`
      * `compute_free_surface`
      * `finalize_water_iteration`
    * `route_sediment`  
  * `apply_subsidence`
  * `finalize_timestep`
  * `output_data`
  * `output_checkpoint`
* `finalize`
---



_pyDeltaRCM_ is built with an object-oriented interface. So, in the most basic use case you "run the model" by 1) initializing the model, and 2) updating the model.

Let's start by importing the package, and then creating a `DeltaModel`.

In [ ]:
import pyDeltaRCM

delta = pyDeltaRCM.DeltaModel()

Instantiating the `DeltaModel()` without any arguments will use a set of [default parameters](https://deltarcm.org/pyDeltaRCM/reference/model/yaml_defaults.html) to configure the model run. The default options are a reasonable set for exploring some controls of the model, and would work perfectly well for a simple demonstration here.



> [Find the full `DeltaModel` object documentation here](https://deltarcm.org/pyDeltaRCM/_autosummary/pyDeltaRCM.model.DeltaModel.html#pyDeltaRCM.model.DeltaModel)

<br>

---

<br>

The delta model is run forward with a call to the `update()` method. So, we simply create a for loop, and call the update function, and then wrap everything up with a call to `finalize()` the model:

In [ ]:
# note: this cell will take ~1 minute to run

for _ in range(0, 5):
    delta.update()

delta.finalize()

That’s it! You ran the *pyDeltaRCM* model for five timesteps, with just five lines of code.

There's not a lot that has happened over these five timesteps though.




In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.imshow(delta.bed_elevation)
plt.show()

 While we have given considerable effort to optimize *pyDeltaRCM*, the model is not "real time" fast. So below, we are going to change some settings from the default values for today's tutorial, so that you will be able to see a delta evolve in real time.

# Inspecting the `DeltaModel` object and Simulating a Mini-Delta

We saw the bed elevation displayed above, but internally this variable is normally referred to as `eta`. Some other important variables names:

* `eta` : bed elevation (m)
* `depth` : flow depth (m)
* `stage` : water surface elevation (m)
* `uw` : along-streamline flow velocity (m/s)
* `qs` : sediment transport volume (m3/s)

Let's see what a few of these look like:

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(delta.eta)  # change this line to view different variables
plt.colorbar(im, ax=ax, shrink=0.6)
plt.show()

To create a smaller delta model that we can see evolve in real time, we will change a number of configuration parameters. We do this by simply passing a keyword and value pair to the model during instantiation.

Any values that are not specified during the model instantiation will take  default values ([default `pyDeltaRCM` model parameters](https://deltarcm.org/pyDeltaRCM/reference/model/yaml_defaults.html)). The input parameter descriptions can be found in the **Attributes** section of the `DeltaModel` documentation [here](https://deltarcm.org/pyDeltaRCM/_autosummary/pyDeltaRCM.model.DeltaModel.html#pydeltarcm-model-deltamodel).

In [ ]:
# create a smaller delta model (default values in [brackets])
delta = pyDeltaRCM.DeltaModel(
    dx=200,                # grid spacing [50 m]
    Np_water=200,          # input water parcels [2000]
    Np_sed=50,             # input sediment parcels [2000]
    C0_percent=0.2,        # input sediment concentation [0.1 %]
    save_eta_grids=True,   # save the topography data [False]
    save_dt=0,             # save interval of 0 seconds [86400]
    clobber_netcdf=True,   # over-write any existing output files [False]
    )

# show the initial condition
fig, ax = plt.subplots()
ax.imshow(delta.eta)
plt.show()


Model parameters can also be defined using a YAML configuration file where the user provides a list of model parameters and their values. For more detailed information on how to write and use YAML files to configure *pyDeltaRCM* model runs, check out [the user guide](https://deltarcm.org/pyDeltaRCM/guides/user_guide.html#configuring-an-input-yaml-file).

In [ ]:
# now loop for 500 timesteps, and plot every 5th step
from IPython.display import clear_output

for i in range(500):
  delta.update()

  if (i % 5) == 0:
    clear_output(wait=True)
    fig, ax = plt.subplots()
    im = ax.imshow(delta.eta)
    plt.colorbar(im, ax=ax, shrink=0.6)
    ax.set_title(i)
    plt.show()

delta.finalize()

#### Reproducible model runs

Compare your model result with your neighbors. Are they the same?

Should they be the same?



In [ ]:
fig, ax = plt.subplots()
ax.imshow(delta.eta)

ax.annotate(
    f'$\\eta={delta.eta[10, 20]:.4f}$',
    xy=(20, 10), xytext=(10, 7.5),
    arrowprops=dict(arrowstyle='-|>',
                    connectionstyle='angle3'),
    bbox=dict(boxstyle='square', fc="w"))

plt.show()

# Analyzing *pyDeltaRCM* model results

*pyDeltaRCM* model outputs are saved to netCDF files. The [netCDF](https://www.unidata.ucar.edu/software/netcdf/) file format is one of several standards for storing scientific data. By saving our model outputs in a standard format, they immediately become interoperable with other scientific software - [QGIS](https://www.qgis.org) for example, can directly read *pyDeltaRCM* output netCDF files.

For more specific details about how the *pyDeltaRCM* output file is formatted, check out the [documentation page](https://deltarcm.org/pyDeltaRCM/info/outputfile.html) on this very subject. In the code block below we will show that the netCDF output file exists and holds data about our model run. 

We will then load and inspect our model output using the `xarray` package which is similar to `numpy` but tracks coordinates and dimensions of data as well (and importantly for us, supports loading of these netCDF files!)

In [ ]:
import os

os.path.isfile('./deltaRCM_Output/pyDeltaRCM_output.nc')

import xarray as xr

ds = xr.load_dataset('./deltaRCM_Output/pyDeltaRCM_output.nc')
ds


We can access a list of the variables which are stored in the netCDF

In [ ]:
list(ds.variables)

If we inspect one of them, we can see the data type (float32), the dimensions of the variable (time, x, y), as well as its units (meters). We also get information about its shape which is organized the same way as the dimensions.

In [ ]:
ds['eta']

Using the netCDF file directly we can plot this variable (the model topography)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 8))

ax[0].imshow(ds['eta'][0, ...])
ax[0].set_title('First Saved Timestep')

ax[1].imshow(ds['eta'][-1, ...])
ax[1].set_title('Final Saved Timestep')

plt.show()

# Load some "full-size" results

We are going to grab some model outputs generated using different input sediment compositions. In the model, this is accomplished by varying the fraction of input sediment that is "sand" (the rest of the sediment is automatically defined as "mud").

<br>

_Warning:_ The next cell might take a minute or two to download the data!

In [ ]:
%%capture

# some pyDeltaRCM model outputs with different input sand fractions
!wget 'https://utexas.box.com/shared/static/tjvr9o7oyt6hthherhaubxlkx6d6e4m7.xz' -O 'pyDeltaRCM_outputs.tar.xz'
!tar -xf pyDeltaRCM_outputs.tar.xz

Now that we've downloaded some pre-run model outputs, we will load them into our notebook. We are going to define a Python "dictionary" which will allow us to access the individual model outputs based on the input sand fraction for each.

Our dictionary will be called `data`. Model outputs were generated with input sand fraction values of 20, 50, and 80%. So we will be able to access each one using the syntax: `data[sf]` where the variable `sf` is either 20, 50, or 80. By setting up dictionary in this way, we can easily change the model output we are visualizing or analyzing in subsequent cells by changing the value of our variable `sf`.

In [ ]:
# access pyDeltaRCM outputs
data = dict()
for sf in [20, 50, 80]:
  # collect the model output
  data[sf] = xr.load_dataset('sandf/sf' + str(sf) + '/pyDeltaRCM_output.nc')

In [ ]:
# pick an output to visualize (options are sf = [20, 50, 80])
# remember sf is the percentage of sand in the input sediment
sf = 20

The below cell plots the topography through time for this model run. If you'd rather visualize the water depths, or the x or y components of the water velocity, just change the variable being visualized from `eta` to the one you wish to view.

In [ ]:
# now loop through topography data to see model evolution
from IPython.display import clear_output

for i in range(data[sf]['eta'].shape[0]):
  # every 25 outputs
  if i % 25 == 0:
    clear_output(wait=True)
    fig, ax = plt.subplots()
    im = ax.imshow(data[sf].eta[i])
    plt.colorbar(im, ax=ax, shrink=0.6)
    ax.set_title('Sand Fraction: ' + str(sf) + '%, Output #: ' + str(i))
    plt.show()

# Some things to ponder

Do environmental conditions (input sediment composition, sea level, vegetation, ice, permafrost) appear to influence the growth and evolution of river deltas?

<br>

Does this reduced-complexity modeling framework seem capable of simulating landscapes that resemble the Wax Lake and Atchafalaya Deltas? What (if anything) does it seem to be missing?